In [1]:
import sys
sys.path.append('/Users/robertcampbell/sqlalchemy-tutorial/')
sys.path
import pytz
from datetime import datetime
from typing import List

from sqlalchemy.sql import text
import pandas as pd
import numpy as np
from numpy.dtypes import DateTime64DType
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score, auc, f1_score, make_scorer, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder
from joblib import load, dump

from schema.schema import Transfer, Event, Fixture, Player
from main import engine

In [2]:
np.random.seed()

# Loading Data

In [3]:
stats_statement = text("SELECT * FROM statistic")
fix_statement = text("SELECT * FROM annotated_fixture")
with engine.connect() as conn:
    stats = conn.execute(stats_statement)
    fixtures = conn.execute(fix_statement)

stats_df = pd.DataFrame(stats)
fix_df = pd.DataFrame(fixtures)

In [4]:
fix_df.set_index("id", inplace=True)
stats_df.set_index("id", inplace=True)
df = stats_df.join(fix_df, "fixture_id")
df['start_time'] = pd.to_datetime(df['start_time'])
df.sort_values('start_time', inplace=True)
df.columns

Index(['team_id', 'player_id', 'fixture_id', 'minutes', 'rating', 'substitute',
       'position', 'goals_total', 'goals_conceded', 'assists', 'saves',
       'total_passes', 'key_passes', 'passing_accuracy', 'tackles', 'blocks',
       'interceptions', 'dribble_attempts', 'successful_dribbles',
       'fouls_drawn', 'penalties_won', 'penalties_commited',
       'penalties_scored', 'penalties_missed', 'penalties_saved', 'captain',
       'total_duels', 'duels_won', 'fouls_committed', 'yellow_cards',
       'red_cards', 'start_time', 'season', 'home_team_id', 'away_team_id',
       'home_name', 'away_name', 'away_team_id:1', 'home_goals', 'away_goals',
       'home_win', 'away_win', 'is_draw'],
      dtype='object')

In [5]:
df.drop(['away_team_id:1'], axis=1, inplace=True)
# for example purposes

In [6]:
cleaned_df = df.copy()
cleaned_df.drop_duplicates('fixture_id', inplace=True)
cleaned_df.sort_values('start_time', inplace=True)
cleaned_df.set_index('fixture_id', inplace=True)

# Basic Cummulative Stats 
- Wins, Losses, Draws
- Goals For, Goals Against

In [7]:
def cumulate(row: pd.Series, col_name, current, historic, n):
    """ cumulate """
    all_fixtures: pd.DataFrame = cleaned_df[cleaned_df[historic] == row[current]]
    res = np.nan
    agg = all_fixtures[all_fixtures["start_time"] < row["start_time"]][col_name]
    if len(agg) > 1:
        res = agg.rolling(n, min_periods=1).sum().shift().iloc[-1]
    return res

In [8]:
RECORD_CUM = 19
STATS_CUM = 38

In [9]:
# Cumulative Wins
cleaned_df['CUM_HT_HW'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_win', current='home_name', historic='home_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_HT_HL'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_win', current='home_name', historic='home_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_HT_HD'] = cleaned_df.apply(lambda row: cumulate(row, col_name='is_draw', current='home_name', historic='home_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_HT_AW'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_win', current='home_name', historic='away_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_HT_AL'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_win', current='home_name', historic='away_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_HT_AD'] = cleaned_df.apply(lambda row: cumulate(row, col_name='is_draw', current='home_name', historic='away_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_AT_HW'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_win', current='away_name', historic='home_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_AT_HL'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_win', current='away_name', historic='home_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_AT_HD'] = cleaned_df.apply(lambda row: cumulate(row, col_name='is_draw', current='away_name', historic='home_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_AT_AW'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_win', current='away_name', historic='away_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_AT_AL'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_win', current='away_name', historic='away_name', n=RECORD_CUM), axis=1)
cleaned_df['CUM_AT_AD'] = cleaned_df.apply(lambda row: cumulate(row, col_name='is_draw', current='away_name', historic='away_name', n=RECORD_CUM), axis=1)
# Cummulative Goals
cleaned_df['H_GF_AH'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_goals', current='home_name', historic='home_name', n=STATS_CUM), axis=1)
cleaned_df['H_GF_OTR'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_goals', current='home_name', historic='away_name', n=STATS_CUM), axis=1)
cleaned_df['H_GA_AH'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_goals', current='home_name', historic='home_name', n=STATS_CUM), axis=1)
cleaned_df['H_GA_OTR'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_goals', current='home_name', historic='away_name', n=STATS_CUM), axis=1)
cleaned_df['A_GF_AH'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_goals', current='away_name', historic='home_name', n=STATS_CUM), axis=1)
cleaned_df['A_GF_OTR'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_goals', current='away_name', historic='away_name', n=STATS_CUM), axis=1)
cleaned_df['A_GA_AH'] = cleaned_df.apply(lambda row: cumulate(row, col_name='away_goals', current='away_name', historic='home_name', n=STATS_CUM), axis=1)
cleaned_df['A_GA_OTR'] = cleaned_df.apply(lambda row: cumulate(row, col_name='home_goals', current='away_name', historic='away_name', n=STATS_CUM), axis=1)

In [10]:
# stats_df.filter(regex='^(?!.*team_id|player|fixture|home|away|season|start_time|is_draw|minutes).*$').columns

# Ratio

In [11]:
def percentage_to_int(val):
    if isinstance(val, str):
        return int(val.replace('%', ''))
    return val

In [12]:
# preprocessing
def extract_ratio_features(dataframe):
    dataframe.loc[:, 'passing_accuracy'] = dataframe['passing_accuracy'].apply(percentage_to_int)
    dataframe.rename(columns={"passing_accuracy": "accurate_passes"}, inplace=True)
    dataframe.loc[:, 'dribble_success_percentage'] = np.round(dataframe['successful_dribbles'] * 100 / dataframe['dribble_attempts'], 3)
    dataframe.loc[:, 'duels_won_percentage'] = np.round(dataframe['duels_won'] * 100 / dataframe['total_duels'], 3)
    dataframe['accurate_passes'].replace(0, np.nan, inplace=True)
    dataframe['total_passes'].replace(0, np.nan, inplace=True)
    dataframe['passing_accuracy'] = np.round(dataframe['accurate_passes'] * 100 / dataframe['total_passes'], 3)

In [13]:
extract_ratio_features(cleaned_df)
extract_ratio_features(df)

/var/folders/nx/l7sd1gss2ksd87brx4b4blxh0000gn/T/ipykernel_70773/207851274.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe['accurate_passes'].replace(0, np.nan, inplace=True)
/var/folders/nx/l7sd1gss2ksd87brx4b4blxh0000gn/T/ipykernel_70773/207851274.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcas

In [14]:
# columns for player statistics
to_remove = ['penalties_won', 'penalties_commited', 'penalties_scored', 'penalties_missed', 'penalties_saved', 'goals_conceded']
sum_cols = ['key_passes', 'total_passes', 'accurate_passes', 'saves', 'fouls_drawn', 'fouls_committed', 'yellow_cards', 'red_cards']
average_cols = ['rating', 'tackles', 'blocks', 'interceptions', 'dribble_success_percentage', 'duels_won_percentage']
max_cols = ['goals_total', 'assists']
indices = ['position', 'team_id']

In [15]:
"""
function average rating
take all games prior to todays game.
take the average rating for all players who are not flagged as a substitute
EOF
TODO: add per position statistics
TODO: number of premier league appearances each player in the XI has had in the past. --> distribution of league experience
TODO: age distribution of the starting XI
TODO: nationality similarity within the team
"""


'\nfunction average rating\ntake all games prior to todays game.\ntake the average rating for all players who are not flagged as a substitute\nEOF\nTODO: add per position statistics\nTODO: number of premier league appearances each player in the XI has had in the past. --> distribution of league experience\nTODO: age distribution of the starting XI\nTODO: nationality similarity within the team\n'

In [16]:
# average stats_window
def get_stats_windows():
    group_cols = ['fixture_id','team_id','substitute']
    current_game = df.iloc[-1]
    grouped: pd.core.groupby.generic.DataFrameGroupBy = df[group_cols + average_cols].groupby(group_cols)
    aggregated: pd.DataFrame = grouped.aggregate(["mean", "median", "min", "max", "std"])
    aggregated.columns = ['_'.join(reversed(col)) for col in aggregated.columns]
    mean_windows = grouped.rolling(STATS_CUM, min_periods=1).aggregate(["mean", "min", "max", "std"]).shift()
    mean_windows.columns = ['_'.join(reversed(col)) for col in mean_windows.columns]
    return mean_windows

In [17]:
mean_windows = get_stats_windows()
mean_windows.xs(867954).xs(33).xs(0)['mean_rating']
mean_windows

mean_rating  min_rating  max_rating  \
fixture_id team_id substitute id                                           
65         33      0          30351          NaN         NaN         NaN   
                              30358          NaN         NaN         NaN   
                              30357          NaN         NaN         NaN   
                              30356          NaN         NaN         NaN   
                              30355          NaN         NaN         NaN   
...                                          ...         ...         ...   
868325     41      1          15160        6.725         6.3         7.2   
                              15161        6.760         6.3         7.2   
                              15163        6.760         6.3         7.2   
                              15164        6.760         6.3         7.2   
                              15162        6.760         6.3         7.2   

                                     std_rating  mean_tackles  min_tackles  \
fixture_id team_id substitute id                                             
65         33      0          30351         NaN           NaN          NaN   
                              30358         NaN      2.000000          2.0   
                              30357         NaN      2.000000          2.0   
                              30356         NaN      2.333333          2.0   
                              30355         NaN      2.000000          1.0   
...                                         ...           ...          ...   
868325     41      1          15160    0.403113      1.000000          1.0   
                              15161    0.357771      1.000000          1.0   
                              15163    0.357771      1.000000          1.0   
                              15164    0.357771      1.000000          1.0   
                              15162    0.357771      1.000000          1.0   

                                     max_tackles  std_tackles  mean_blocks  \
fixture_id team_id substitute id                                             
65         33      0          30351          NaN          NaN          NaN   
                              30358          2.0          NaN          0.0   
                              30357          2.0     0.000000          0.0   
                              30356          3.0     0.577350          0.0   
                              30355          3.0     0.816497          0.0   
...                                          ...          ...          ...   
868325     41      1          15160          1.0     0.000000          NaN   
                              15161          1.0     0.000000          NaN   
                              15163          1.0     0.000000          NaN   
                              15164          1.0     0.000000          NaN   
                              15162          1.0     0.000000          NaN   

                                     min_blocks  ...  max_interceptions  \
fixture_id team_id substitute id                 ...                      
65         33      0          30351         NaN  ...                NaN   
                              30358         0.0  ...                1.0   
                              30357         0.0  ...                1.0   
                              30356         0.0  ...                1.0   
                              30355         0.0  ...                3.0   
...                                         ...  ...                ...   
868325     41      1          15160         NaN  ...                1.0   
                              15161         NaN  ...                1.0   
                              15163         NaN  ...                1.0   
                              15164         NaN  ...                1.0   
                              15162         NaN  ...                1.0   

                                     std_i

In [18]:
def get_stats(row):
    """ For every col in average_cols grab the average mean, std, min and max """
    fixture_id = row.name
    home_id, away_id = row.loc['home_team_id'], row.loc['away_team_id']
    
    home_stats = mean_windows.xs(fixture_id).xs(home_id).xs(0).mean()
    home_stats.index = ["home_" + name for name in home_stats.index]
    
    away_stats = mean_windows.xs(fixture_id).xs(away_id).xs(0).mean()
    away_stats.index = ["away_" + name for name in home_stats.index]
    
    return pd.concat([home_stats, away_stats])

In [19]:
mean_stats = cleaned_df.apply(get_stats, axis=1, result_type="expand")
cleaned_df = pd.concat([cleaned_df, mean_stats], axis=1)

In [20]:
df.isna().sum().sort_values(ascending=False)[:12]

penalties_scored              97530
penalties_commited            97530
penalties_missed              97530
penalties_saved               97530
penalties_won                 97530
assists                       92157
saves                         91572
goals_total                   90288
rating                        86457
dribble_success_percentage    59183
tackles                       48692
blocks                        40138
dtype: int64

In [21]:
# drop penalties columns as they are all null
cleaned_df.drop(columns=["penalties_scored", "penalties_commited", "penalties_missed", "penalties_won"], inplace=True)

In [22]:
df.set_index('fixture_id', inplace=True)

In [23]:
def return_same(val):
    return int(val.iloc[0])

In [24]:
sum_cols + max_cols

['key_passes',
 'total_passes',
 'accurate_passes',
 'saves',
 'fouls_drawn',
 'fouls_committed',
 'yellow_cards',
 'red_cards',
 'goals_total',
 'assists']

# Get Top Goal Scorers

In [25]:
def get_top_player_stats():
    agg_cols = sum_cols + max_cols
    agg_functions = {k: 'sum' for k in agg_cols}
    agg_functions['team_id'] = return_same
    groupby_player = df[df["substitute"] == 0][['player_id', 'team_id'] + sum_cols + max_cols].groupby(['player_id'], as_index=False)
    top_player_statistics = groupby_player.rolling(STATS_CUM, min_periods=1).aggregate(agg_functions)
    return top_player_statistics
top_player_stats = get_top_player_stats()

In [68]:
top_player_stats

key_passes  total_passes  accurate_passes  saves  \
player_id fixture_id                                                     
0         79                 0.0          46.0             95.0    NaN   
5         868025             NaN          91.0             88.0    NaN   
          868033             NaN         164.0            159.0    NaN   
          868042             NaN         243.0            230.0    NaN   
          868051             NaN         297.0            278.0    NaN   
...                          ...           ...              ...    ...   
311334    868219             2.0          57.0             44.0    NaN   
          868287             2.0          73.0             57.0    NaN   
          868193             2.0          88.0             68.0    NaN   
          868259             3.0         113.0             88.0    NaN   
          868317             3.0         122.0             94.0    NaN   

                      fouls_drawn  fouls_committed  yellow_cards  red_cards  \
player_id fixture_id                                                          
0         79                  0.0              2.0           0.0        0.0   
5         868025              NaN              2.0           0.0        0.0   
          868033              1.0              2.0           0.0        0.0   
          868042              1.0              2.0           0.0        0.0   
          868051              1.0              5.0           1.0        0.0   
...                           ...              ...           ...        ...   
311334    868219              4.0              1.0           0.0        0.0   
          868287              4.0              4.0           0.0        0.0   
          868193              5.0              4.0           0.0        0.0   
          868259              6.0              5.0           0.0        0.0   
          868317              7.0              6.0           1.0        0.0   

                      goals_total  assists  team_id  
player_id fixture_id                                 
0         79                  NaN      NaN     36.0  
5         868025              NaN      NaN     50.0  
          868033              NaN      NaN     50.0  
          868042              NaN      NaN     50.0  
          868051              NaN      NaN     50.0  
...                           ...      ...      ...  
311334    868219              1.0      NaN     51.0  
          868287              1.0      NaN     51.0  
          868193              1.0      NaN     51.0  
          868259              1.0      NaN     51.0  
          868317              1.0      NaN     51.0  

[72300 rows x 11 columns]

In [26]:
def get_top_n_stats(row, stat, team, n):
    """ get_top_n_goal_scorers """
    vals = top_player_stats[top_player_stats["team_id"] == row[team]].xs(row.name, level=1)[stat].nlargest(n).sort_values(ascending=False)
    top_goal_scorers = pd.Series(vals.values)
    return top_goal_scorers

In [27]:
# top goal scorers
top_home_scorers_df = cleaned_df.apply(lambda x: get_top_n_stats(x, 'goals_total', 'home_team_id', 3), axis=1, result_type="expand")
top_home_scorers_df.columns = ["home_top_goal_scorer", "home_top_goal_scorer_2", "home_top_goal_scorer_3"]
cleaned_df = pd.concat([cleaned_df, top_home_scorers_df], axis=1)

top_away_scorers_df = cleaned_df.apply(lambda x: get_top_n_stats(x, 'goals_total', 'away_team_id', 3), axis=1, result_type="expand")
top_away_scorers_df.columns = ["away_top_goal_scorer", "away_top_goal_scorer_2", "away_top_goal_scorer_3"]
cleaned_df = pd.concat([cleaned_df, top_away_scorers_df], axis=1)

# top assisters
top_home_assisters_df = cleaned_df.apply(lambda x: get_top_n_stats(x, 'assists', 'home_team_id', 3), axis=1, result_type="expand")
top_home_assisters_df.columns = ["home_top_assister", "home_top_assister_2", "home_top_assister_3"]
cleaned_df = pd.concat([cleaned_df, top_home_assisters_df], axis=1)

top_away_assisters_df = cleaned_df.apply(lambda x: get_top_n_stats(x, 'assists', 'away_team_id', 3), axis=1, result_type="expand")
top_away_assisters_df.columns = ["away_top_assister", "away_top_assister_2", "away_top_assister_3"]
cleaned_df = pd.concat([cleaned_df, top_away_assisters_df], axis=1)

# Get Cummulative stats

In [28]:
def agg_sums(row, team, stat):
    return top_player_stats[top_player_stats["team_id"] == row[team]].xs(row.name, level=1)[stat].sum()
top_player_stats.xs(868219, level=1)
cleaned_df["home_team_id"]

fixture_id
192904    49
192905    42
192966    76
192297    33
192300    46
          ..
868325    41
868317    66
868318    55
868316    42
868319    49
Name: home_team_id, Length: 3042, dtype: int64

In [29]:
cleaned_df["home_team_cummulative_saves"] = cleaned_df.apply(lambda x: agg_sums(x, "home_team_id", "saves"), axis=1)
cleaned_df["away_team_cummulative_saves"] = cleaned_df.apply(lambda x: agg_sums(x, "away_team_id", "saves"), axis=1)
cleaned_df["home_team_cummulative_yellows"] = cleaned_df.apply(lambda x: agg_sums(x, "home_team_id", "yellow_cards"), axis=1)
cleaned_df["home_team_cummulative_yellows"] = cleaned_df.apply(lambda x: agg_sums(x, "away_team_id", "yellow_cards"), axis=1)
cleaned_df["home_team_cummulative_reds"] = cleaned_df.apply(lambda x: agg_sums(x, "home_team_id", "red_cards"), axis=1)
cleaned_df["away_team_cummulative_reds"] = cleaned_df.apply(lambda x: agg_sums(x, "away_team_id", "red_cards"), axis=1)
cleaned_df["home_team_cummulative_fouls_committed"] = cleaned_df.apply(lambda x: agg_sums(x, "home_team_id", "fouls_committed"), axis=1)
cleaned_df["away_team_cummulative_fouls_committed"] = cleaned_df.apply(lambda x: agg_sums(x, "away_team_id", "fouls_committed"), axis=1)
cleaned_df["home_team_cummulative_fouls_drawn"] = cleaned_df.apply(lambda x: agg_sums(x, "home_team_id", "fouls_drawn"), axis=1)
cleaned_df["away_team_cummulative_fouls_drawn"] = cleaned_df.apply(lambda x: agg_sums(x, "away_team_id", "fouls_drawn"), axis=1)
cleaned_df["home_team_cummulative_key_passes"] = cleaned_df.apply(lambda x: agg_sums(x, "home_team_id" ,"key_passes"), axis=1)
cleaned_df["away_team_cummulative_key_passes"] = cleaned_df.apply(lambda x: agg_sums(x, "away_team_id" ,"key_passes"), axis=1)

## Get Club History

In [30]:
cleaned_df['away_name'].unique()

array(['Manchester City', 'Aston Villa', 'Liverpool', 'Tottenham',
       'Sunderland', 'Watford', 'Crystal Palace', 'Swansea',
       'Southampton', 'West Ham', 'Manchester United', 'Everton',
       'Stoke City', 'Norwich', 'West Brom', 'Newcastle', 'Leicester',
       'Arsenal', 'Chelsea', 'Bournemouth', 'Hull City', 'Middlesbrough',
       'Burnley', 'Huddersfield', 'Brighton', 'Cardiff', 'Fulham',
       'Wolves', 'Sheffield Utd', 'Leeds', 'Brentford',
       'Nottingham Forest'], dtype=object)

In [31]:
def get_winner_id(row):
    """ GET WINNER """
    if row["home_win"] == 1:
        return row["home_team_id"]
    elif row["away_win"] == 1:
        return row["away_team_id"]
    else:
        return np.nan

In [32]:
cleaned_df["winner_id"] = cleaned_df.apply(get_winner_id, axis=1)

In [33]:
def get_club_history(row, n):
    """ get home_wins, away_wins, draw between two current clubs for the last n games """
    ch = cleaned_df[
        ((cleaned_df["home_team_id"] == row["away_team_id"]) & (cleaned_df["away_team_id"] == row["home_team_id"])) | 
        ((cleaned_df["home_team_id"] == row["home_team_id"]) & (cleaned_df["away_team_id"] == row["away_team_id"]))
    ]
    history = ch[ch["start_time"] < row["start_time"]]
    home_wins = (history["winner_id"] == row["home_team_id"]).sum()
    away_wins = (history["winner_id"] == row["away_team_id"]).sum()
    draws = history["winner_id"].isna().sum()
    res = pd.Series([home_wins, away_wins, draws], index=["club_history_home_wins", "club_history_away_wins", "club_history_draws"])
    return res

In [34]:
club_history_df = cleaned_df.apply(lambda x: get_club_history(x, 12), axis=1, result_type="expand")
cleaned_df = pd.concat([cleaned_df, club_history_df], axis=1)

In [35]:
cleaned_df.columns[:60]

Index(['team_id', 'player_id', 'minutes', 'rating', 'substitute', 'position',
       'goals_total', 'goals_conceded', 'assists', 'saves', 'total_passes',
       'key_passes', 'accurate_passes', 'tackles', 'blocks', 'interceptions',
       'dribble_attempts', 'successful_dribbles', 'fouls_drawn',
       'penalties_saved', 'captain', 'total_duels', 'duels_won',
       'fouls_committed', 'yellow_cards', 'red_cards', 'start_time', 'season',
       'home_team_id', 'away_team_id', 'home_name', 'away_name', 'home_goals',
       'away_goals', 'home_win', 'away_win', 'is_draw', 'CUM_HT_HW',
       'CUM_HT_HL', 'CUM_HT_HD', 'CUM_HT_AW', 'CUM_HT_AL', 'CUM_HT_AD',
       'CUM_AT_HW', 'CUM_AT_HL', 'CUM_AT_HD', 'CUM_AT_AW', 'CUM_AT_AL',
       'CUM_AT_AD', 'H_GF_AH', 'H_GF_OTR', 'H_GA_AH', 'H_GA_OTR', 'A_GF_AH',
       'A_GF_OTR', 'A_GA_AH', 'A_GA_OTR', 'dribble_success_percentage',
       'duels_won_percentage', 'passing_accuracy'],
      dtype='object')

# Remove Redundant features

In [36]:
# TODO: remove right after extracting them.
to_remove = [
    'home_team_id', 'away_team_id', 'fouls_committed', 
    'fouls_drawn', 'yellow_cards', 'red_cards', 
    'season', 'team_id', 'player_id', 'minutes', 
    'substitute', 'position', 'goals_total',
    'goals_conceded', 'assists', 'saves', 
    'key_passes', 'total_passes', 'accurate_passes',
    'tackles', 'blocks', 'interceptions', 'dribble_attempts',
    'successful_dribbles', 'captain', 'total_duels', 'duels_won',
    'home_win', 'away_win', 'is_draw', 'penalties_saved'
]
cleaned_df.drop(to_remove, axis=1, inplace=True)

In [37]:
cleaned_df.to_csv('../data/prem_data.csv')

# Get Naive Prediction

In [38]:
set_config(transform_output="pandas")
DRAW_THRESHOLD = 0.5

In [39]:
# detect continuous, categorical, and date features
numeric_cols = cleaned_df.describe().columns.to_list()
numeric_cols.pop(numeric_cols.index('home_goals'))
numeric_cols.pop(numeric_cols.index('away_goals'))
date_cols = [numeric_cols.pop(numeric_cols.index('start_time'))]

In [40]:
X = cleaned_df.drop(['home_goals', 'away_goals'], axis=1)
home_goals = cleaned_df['home_goals'].to_numpy()
away_goals = cleaned_df['away_goals'].to_numpy()
X_train, X_test, y_home_train, y_home_test, y_away_train, y_away_test = train_test_split(X, home_goals, away_goals, test_size=0.01, shuffle=False)

In [41]:
X_test

,rating,start_time,home_name,away_name,CUM_HT_HW,CUM_HT_HL,CUM_HT_HD,CUM_HT_AW,CUM_HT_AL,CUM_HT_AD,...,home_team_cummulative_fouls_committed,away_team_cummulative_fouls_committed,home_team_cummulative_fouls_drawn,away_team_cummulative_fouls_drawn,home_team_cummulative_key_passes,away_team_cummulative_key_passes,winner_id,club_history_home_wins,club_history_away_wins,club_history_draws
fixture_id,,,,,,,,,,,,,,,,,,,,,
868300,NaN,2023-05-13 10:00:00,Crystal Palace,Bournemouth,7.0,5.0,7.0,4.0,10.0,5.0,...,329.0,221.0,472.0,209.0,306.0,132.0,52.0,5,2,4
868299,6.7,2023-05-13 10:00:00,Chelsea,Nottingham Forest,7.0,5.0,7.0,5.0,9.0,5.0,...,128.0,154.0,138.0,136.0,92.0,45.0,NaN,0,0,1
868304,NaN,2023-05-13 10:00:00,Manchester United,Wolves,13.0,1.0,5.0,7.0,9.0,3.0,...,260.0,192.0,226.0,171.0,358.0,167.0,33.0,4,2,3
868305,7.6,2023-05-13 10:00:00,Southampton,Fulham,3.0,12.0,4.0,4.0,13.0,2.0,...,195.0,212.0,217.0,223.0,161.0,162.0,36.0,2,2,1
868301,6.2,2023-05-14 09:00:00,Everton,Manchester City,7.0,9.0,3.0,1.0,9.0,9.0,...,252.0,241.0,179.0,229.0,156.0,325.0,50.0,1,10,4
868298,NaN,2023-05-14 09:00:00,Brentford,West Ham,8.0,3.0,8.0,5.0,7.0,7.0,...,177.0,151.0,218.0,118.0,196.0,109.0,55.0,3,0,0
868296,6.2,2023-05-14 11:30:00,Arsenal,Brighton,15.0,1.0,3.0,11.0,5.0,3.0,...,213.0,222.0,274.0,231.0,342.0,266.0,51.0,4,4,3
868303,6.2,2023-05-15 15:00:00,Leicester,Liverpool,6.0,10.0,3.0,5.0,11.0,3.0,...,373.0,296.0,337.0,216.0,278.0,353.0,40.0,4,10,1
868193,6.5,2023-05-18 14:30:00,Newcastle,Brighton,12.0,2.0,5.0,9.0,4.0,6.0,...,388.0,225.0,337.0,192.0,344.0,248.0,34.0,1,4,6


In [42]:
# create custom transformer
weekdays = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

class DateTransformer(BaseEstimator, TransformerMixin):
    """ Date Transformer """
    def fit(self, X, y=None):
        """ fit """
        return self
    
    def to_weekday(self, date:datetime) -> int:
        """ RETURN weekday as a string """
        return weekdays[date.weekday()].lower()
    
    def is_weekend(self, date:datetime) -> int:
        """ RETURN 1 if its a weekend and 0 otherwise"""
        if date.weekday() >= 5:
            return 1
        return 0
    
    def time_of_day_classification(self, date:datetime) -> int:
        """TODO implement time of day classification: Morning, Midday, Night """
        pass

    def transform(self, X:pd.DataFrame):
        """ transform """
        types = pd.Series(X.dtypes.apply(lambda x: str(x)))
        date_columns = types[types.str.contains('datetime')].index.to_list()

        return_cols = []
        for date_col in date_columns:
            # applying functions
            X[f"{date_col}_weekday"] = X[date_col].apply(self.to_weekday)
            X[f"{date_col}_is_weekend"] = X[date_col].apply(self.is_weekend)

            # add to return cols list
            return_cols.append(f"{date_col}_weekday")
            return_cols.append(f"{date_col}_is_weekend")
            
        return X.loc[:, return_cols]

In [43]:
mae_score = make_scorer(mean_absolute_error)

parameters = {
    'alpha': [0.03, 0.05, 0.1],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7]
}

date_transformer = Pipeline(steps=[
    ('transformer', DateTransformer()),
    ('encoder', OneHotEncoder(use_cat_names=True))])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

feature_preprocessor = ColumnTransformer(transformers=[
    ('numerical', numeric_transformer, numeric_cols),
    ('datetime', date_transformer, date_cols)
])

home_pipeline = Pipeline(steps=[
    ('preprocessor', feature_preprocessor),
    ('regressor', GridSearchCV(ElasticNet(), param_grid=parameters, refit=True, scoring=mae_score))
])

away_pipeline = Pipeline(steps=[
    ('preprocessor', feature_preprocessor),
    ('regressor', GridSearchCV(ElasticNet(), param_grid=parameters, refit=True, scoring=mae_score))
])

In [44]:
feature_preprocessor.fit_transform(X_train)

,numerical__rating,numerical__CUM_HT_HW,numerical__CUM_HT_HL,numerical__CUM_HT_HD,numerical__CUM_HT_AW,numerical__CUM_HT_AL,numerical__CUM_HT_AD,numerical__CUM_AT_HW,numerical__CUM_AT_HL,numerical__CUM_AT_HD,...,numerical__club_history_away_wins,numerical__club_history_draws,datetime__start_time_weekday_saturday,datetime__start_time_weekday_sunday,datetime__start_time_weekday_monday,datetime__start_time_weekday_friday,datetime__start_time_weekday_tuesday,datetime__start_time_weekday_wednesday,datetime__start_time_weekday_thursday,datetime__start_time_is_weekend
fixture_id,,,,,,,,,,,,,,,,,,,,,
192904,0.011494,-0.196405,-0.097403,-0.032720,-0.188052,0.153230,-0.028237,-0.192420,-0.105033,-0.035929,...,-0.858459,-0.803645,1,0,0,0,0,0,0,1
192905,0.011494,-0.196405,-0.097403,-0.032720,-0.188052,0.153230,-0.028237,-0.192420,-0.105033,-0.035929,...,-0.858459,-0.803645,0,1,0,0,0,0,0,1
192966,0.011494,-0.196405,-0.097403,-0.032720,-0.188052,0.153230,-0.028237,-0.192420,-0.105033,-0.035929,...,-0.858459,-0.803645,0,0,1,0,0,0,0,0
192297,0.011494,-0.196405,-0.097403,-0.032720,-0.188052,0.153230,-0.028237,-0.192420,-0.105033,-0.035929,...,-0.858459,-0.803645,1,0,0,0,0,0,0,1
192300,0.011494,-0.196405,-0.097403,-0.032720,-0.188052,0.153230,-0.028237,-0.192420,-0.105033,-0.035929,...,-0.858459,-0.803645,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868288,-2.460632,-0.196405,0.912749,-0.032720,0.100103,0.698268,-0.546663,-0.449122,1.562972,-0.518880,...,0.109820,-0.101854,0,0,1,0,0,0,0,0
868287,1.865590,0.574619,-0.434120,0.452606,0.964567,-0.664328,0.490189,-0.192420,1.229371,-0.518880,...,1.562237,0.599937,0,0,1,0,0,0,0,0
868292,-1.842601,-0.710421,-0.097403,0.937932,-1.340671,1.243306,-0.546663,-1.219229,2.230174,-0.035929,...,-0.858459,-0.803645,0,0,1,0,0,0,0,0


In [45]:
# FIT model
home_pipeline.fit(X_train, y_home_train)
away_pipeline.fit(X_train, y_away_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['rating', 'CUM_HT_HW',
                                                   'CUM_HT_HL', 'CUM_HT_HD',
                                                   'CUM_HT_AW', 'CUM_HT_AL',
                                                   'CUM_HT_AD', 'CUM_AT_HW',
                                                   'CUM_AT_HL', 'CUM_AT_HD',
                                                   'CUM_AT_AW', 'CUM_AT_AL',
                                                   'CUM_AT_AD', 'H_GF_AH',
                                                   'H_GF_OTR', 'H_GA_A...
                                                   'home_mean_tackles',
                                                   'home_min_tackles', ...]),
                                                 ('datetime',
                                                  Pipeline(steps=[('transformer',
                                                                   DateTransformer()),
                                                                  ('encoder',
                                                                   OneHotEncoder(use_cat_names=True))]),
                                                  ['start_time'])])),
                ('regressor',
                 GridSearchCV(estimator=ElasticNet(),
                              param_grid={'alpha': [0.03, 0.05, 0.1],
                                          'l1_ratio': [0.1, 0.3, 0.5, 0.7]},
                              scoring=make_scorer(mean_absolute_error, response_method='predict')))])

In [46]:
# Eval on test set
y_home_pred = home_pipeline.predict(X_test)
y_away_pred = away_pipeline.predict(X_test)
def print_results():
    print(f"HOME TRAIN MAE: {mean_absolute_error(home_pipeline.predict(X_train), y_home_train)}")
    print(f"AWAY TRAIN MAE: {mean_absolute_error(home_pipeline.predict(X_train), y_away_train)}")
    print(f"HOME TEST MAE: {mean_absolute_error(y_home_pred.round(), y_home_test)}")
    print(f"AWAY TEST MAE: {mean_absolute_error(y_away_pred.round(), y_away_test)}")

In [47]:
X_test

,rating,start_time,home_name,away_name,CUM_HT_HW,CUM_HT_HL,CUM_HT_HD,CUM_HT_AW,CUM_HT_AL,CUM_HT_AD,...,home_team_cummulative_fouls_committed,away_team_cummulative_fouls_committed,home_team_cummulative_fouls_drawn,away_team_cummulative_fouls_drawn,home_team_cummulative_key_passes,away_team_cummulative_key_passes,winner_id,club_history_home_wins,club_history_away_wins,club_history_draws
fixture_id,,,,,,,,,,,,,,,,,,,,,
868300,NaN,2023-05-13 10:00:00,Crystal Palace,Bournemouth,7.0,5.0,7.0,4.0,10.0,5.0,...,329.0,221.0,472.0,209.0,306.0,132.0,52.0,5,2,4
868299,6.7,2023-05-13 10:00:00,Chelsea,Nottingham Forest,7.0,5.0,7.0,5.0,9.0,5.0,...,128.0,154.0,138.0,136.0,92.0,45.0,NaN,0,0,1
868304,NaN,2023-05-13 10:00:00,Manchester United,Wolves,13.0,1.0,5.0,7.0,9.0,3.0,...,260.0,192.0,226.0,171.0,358.0,167.0,33.0,4,2,3
868305,7.6,2023-05-13 10:00:00,Southampton,Fulham,3.0,12.0,4.0,4.0,13.0,2.0,...,195.0,212.0,217.0,223.0,161.0,162.0,36.0,2,2,1
868301,6.2,2023-05-14 09:00:00,Everton,Manchester City,7.0,9.0,3.0,1.0,9.0,9.0,...,252.0,241.0,179.0,229.0,156.0,325.0,50.0,1,10,4
868298,NaN,2023-05-14 09:00:00,Brentford,West Ham,8.0,3.0,8.0,5.0,7.0,7.0,...,177.0,151.0,218.0,118.0,196.0,109.0,55.0,3,0,0
868296,6.2,2023-05-14 11:30:00,Arsenal,Brighton,15.0,1.0,3.0,11.0,5.0,3.0,...,213.0,222.0,274.0,231.0,342.0,266.0,51.0,4,4,3
868303,6.2,2023-05-15 15:00:00,Leicester,Liverpool,6.0,10.0,3.0,5.0,11.0,3.0,...,373.0,296.0,337.0,216.0,278.0,353.0,40.0,4,10,1
868193,6.5,2023-05-18 14:30:00,Newcastle,Brighton,12.0,2.0,5.0,9.0,4.0,6.0,...,388.0,225.0,337.0,192.0,344.0,248.0,34.0,1,4,6


In [48]:
# GETTING ACCURACY SCORE FOR CLASSIFICATION --> TODO
def get_results(home_array: List[float], away_array: List[float]) -> List[float]:
    """ get results """
    results_array = []
    for home_pred, away_pred in zip(home_array, away_array):
        diff = home_pred - away_pred
        if abs(diff) <= DRAW_THRESHOLD: # draw
            results_array.append(0)
        elif diff > 0: # home team winds
            results_array.append(1)
        else: # away team wins
            results_array.append(2)
    return results_array

# create win, loss, draw array
pred_results = get_results(home_array=y_home_pred, away_array=y_away_pred)
actual_results = get_results(home_array=y_home_test, away_array=y_away_test)
accuracy_score(pred_results, actual_results)
# f1_score(actual_results, pred_results, average='macro')

0.7419354838709677

In [49]:
# GET FEATURE IMPORTANCES
def get_feature_importances(pipeline, X_test):
    regressor = pipeline['regressor'].best_estimator_
    coefs = list(np.round(regressor.coef_, 3))
    features = pipeline['preprocessor'].transform(X_test).columns.to_list()
    zipped = list(zip(coefs, features))
    feature_importances = sorted(zipped, key=lambda x: abs(x[0]), reverse=True)
    return feature_importances

In [50]:
home_features = get_feature_importances(home_pipeline, X_test)
away_features = get_feature_importances(away_pipeline, X_test)

In [51]:
print(home_features[:8])
print(away_features[:8])

[(-0.148, 'numerical__away_home_min_rating'), (0.129, 'numerical__home_top_assister_2'), (-0.101, 'numerical__away_home_mean_rating'), (-0.07, 'numerical__CUM_AT_AW'), (0.066, 'numerical__home_top_assister'), (0.062, 'numerical__CUM_HT_AW'), (0.062, 'numerical__home_top_assister_3'), (-0.05, 'numerical__CUM_HT_HL')]
[(-0.179, 'numerical__home_mean_rating'), (0.108, 'numerical__away_team_cummulative_key_passes'), (0.105, 'numerical__away_home_max_rating'), (0.079, 'numerical__away_top_goal_scorer'), (-0.077, 'datetime__start_time_weekday_monday'), (0.076, 'numerical__away_top_assister_2'), (0.069, 'numerical__home_std_rating'), (-0.067, 'numerical__H_GF_OTR')]


# Create Hyperparameter Table


In [52]:
# home team and away team
# model_info: serialized_pipeline, feature_importances
# metrics: mean absolute error, r2, adjusted_r2

# same dataset for both
# training_methodology: training_set, test_set learning (batch, online)

In [53]:
home_features

[(-0.148, 'numerical__away_home_min_rating'),
 (0.129, 'numerical__home_top_assister_2'),
 (-0.101, 'numerical__away_home_mean_rating'),
 (-0.07, 'numerical__CUM_AT_AW'),
 (0.066, 'numerical__home_top_assister'),
 (0.062, 'numerical__CUM_HT_AW'),
 (0.062, 'numerical__home_top_assister_3'),
 (-0.05, 'numerical__CUM_HT_HL'),
 (-0.041, 'numerical__A_GF_AH'),
 (0.028, 'numerical__home_top_goal_scorer'),
 (0.028, 'numerical__home_top_goal_scorer_3'),
 (-0.024, 'numerical__away_top_assister_3'),
 (0.017, 'numerical__home_max_rating'),
 (-0.012, 'numerical__away_top_goal_scorer_3'),
 (-0.009, 'numerical__CUM_HT_AL'),
 (-0.006, 'numerical__away_home_min_duels_won_percentage'),
 (-0.004, 'numerical__CUM_AT_HW'),
 (-0.004, 'numerical__away_top_goal_scorer'),
 (-0.003, 'numerical__away_home_mean_duels_won_percentage'),
 (-0.0, 'numerical__rating'),
 (0.0, 'numerical__CUM_HT_HW'),
 (-0.0, 'numerical__CUM_HT_HD'),
 (-0.0, 'numerical__CUM_HT_AD'),
 (0.0, 'numerical__CUM_AT_HL'),
 (-0.0, 'numerical__

In [54]:
arrays = [
    ["model_info", "model_info", "model_info", "metrics", "metrics", "metrics", "data", "data", "data"],
    ["model_path", "alpha", "l1_ratio", "mae", "r2", "adjusted_r2", "training_set_path", "test_set_path", "learning_method"]
]
tuples = list(zip(*arrays))
col_index = pd.MultiIndex.from_tuples(tuples)
col_index

MultiIndex([('model_info',        'model_path'),
            ('model_info',             'alpha'),
            ('model_info',          'l1_ratio'),
            (   'metrics',               'mae'),
            (   'metrics',                'r2'),
            (   'metrics',       'adjusted_r2'),
            (      'data', 'training_set_path'),
            (      'data',     'test_set_path'),
            (      'data',   'learning_method')],
           )

In [55]:
home_ht = pd.DataFrame(columns=["model_path", "alpha", "l1_ratio", "mae", "r2", "adjusted_r2", "dataset_path"])
away_ht = pd.DataFrame(columns=["model_path", "alpha", "l1_ratio", "mae", "r2", "adjusted_r2", "dataset_path"])

In [56]:
[
    home_pipeline['regressor'].best_estimator_.alpha,
    home_pipeline['regressor'].best_estimator_.l1_ratio,
]

[0.1, 0.7]

In [57]:
dump(home_pipeline, "../data/home_pipeline.joblib")

['../data/home_pipeline.joblib']

In [58]:
loaded_pipeline = load("../data/home_pipeline.joblib")

In [59]:
loaded_home_preds = loaded_pipeline.predict(X_test)

In [60]:
print(f"HOME TEST MAE: {mean_absolute_error(loaded_home_preds.round(), y_home_test)}")

HOME TEST MAE: 0.5483870967741935


In [61]:
home_ht

,model_path,alpha,l1_ratio,mae,r2,adjusted_r2,dataset_path


In [62]:
trial_results = pd.DataFrame({
    'model_path': ['path/to/model'], 
    'alpha': [0.1], 
    'l1_ratio': [0.03], 
    'mae': [0.95],
    'dataset_path': '/data/prem_data.csv'
})

In [63]:
home_ht = pd.concat([home_ht, trial_results])
home_ht

/var/folders/nx/l7sd1gss2ksd87brx4b4blxh0000gn/T/ipykernel_70773/958845530.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  home_ht = pd.concat([home_ht, trial_results])


,model_path,alpha,l1_ratio,mae,r2,adjusted_r2,dataset_path
0,path/to/model,0.1,0.03,0.95,NaN,NaN,/data/prem_data.csv


In [64]:
X_test

,rating,start_time,home_name,away_name,CUM_HT_HW,CUM_HT_HL,CUM_HT_HD,CUM_HT_AW,CUM_HT_AL,CUM_HT_AD,...,home_team_cummulative_fouls_committed,away_team_cummulative_fouls_committed,home_team_cummulative_fouls_drawn,away_team_cummulative_fouls_drawn,home_team_cummulative_key_passes,away_team_cummulative_key_passes,winner_id,club_history_home_wins,club_history_away_wins,club_history_draws
fixture_id,,,,,,,,,,,,,,,,,,,,,
868300,NaN,2023-05-13 10:00:00,Crystal Palace,Bournemouth,7.0,5.0,7.0,4.0,10.0,5.0,...,329.0,221.0,472.0,209.0,306.0,132.0,52.0,5,2,4
868299,6.7,2023-05-13 10:00:00,Chelsea,Nottingham Forest,7.0,5.0,7.0,5.0,9.0,5.0,...,128.0,154.0,138.0,136.0,92.0,45.0,NaN,0,0,1
868304,NaN,2023-05-13 10:00:00,Manchester United,Wolves,13.0,1.0,5.0,7.0,9.0,3.0,...,260.0,192.0,226.0,171.0,358.0,167.0,33.0,4,2,3
868305,7.6,2023-05-13 10:00:00,Southampton,Fulham,3.0,12.0,4.0,4.0,13.0,2.0,...,195.0,212.0,217.0,223.0,161.0,162.0,36.0,2,2,1
868301,6.2,2023-05-14 09:00:00,Everton,Manchester City,7.0,9.0,3.0,1.0,9.0,9.0,...,252.0,241.0,179.0,229.0,156.0,325.0,50.0,1,10,4
868298,NaN,2023-05-14 09:00:00,Brentford,West Ham,8.0,3.0,8.0,5.0,7.0,7.0,...,177.0,151.0,218.0,118.0,196.0,109.0,55.0,3,0,0
868296,6.2,2023-05-14 11:30:00,Arsenal,Brighton,15.0,1.0,3.0,11.0,5.0,3.0,...,213.0,222.0,274.0,231.0,342.0,266.0,51.0,4,4,3
868303,6.2,2023-05-15 15:00:00,Leicester,Liverpool,6.0,10.0,3.0,5.0,11.0,3.0,...,373.0,296.0,337.0,216.0,278.0,353.0,40.0,4,10,1
868193,6.5,2023-05-18 14:30:00,Newcastle,Brighton,12.0,2.0,5.0,9.0,4.0,6.0,...,388.0,225.0,337.0,192.0,344.0,248.0,34.0,1,4,6
